# App Store

This notebook is intended to help you get started with managing apps and app instances in the H2O AI Cloud.

* **Product Documention:** https://h2oai.github.io/h2o-ai-cloud/ 

**WARNING:** Please note that this API is considered alpha and can be expected to have breaking changes. 

## Prerequistites

This tutorial replies on the alpha App Store API which can be installed into a python environment by:
`pip install https://h2o-public-test-data.s3.amazonaws.com/e2e-testing/appstore-0.8.0-py3-none-any.whl`


We also set the following variables to connect to a specific H2O AI Cloud environment. They can be found by logging into the platform, clicking on your name, and choosing the CLI & API Access page. Then, copy values from the Configuring the H2O CLI section.

In [12]:
OIDC_ADDRESS = 'https://auth.demo.h2o.ai/auth/realms/q8s-internal'
CLIENT_ID = 'q8s-internal-platform'
REFRESH_TOKEN = 'https://cloud-internal.h2o.ai/auth/get-platform-token'
ENDPOINT = 'https://cloud-internal.h2o.ai'

In [13]:
from getpass import getpass
from datetime import datetime

import appstore

import pandas as pd

## Securely Connect

We connect to the App Store, and then create a service object for interacting with App and Tag objects.

In [20]:
print(f"Visit {REFRESH_TOKEN} to get your personal access token")
cp = appstore.TokenContextProvider(
    oidc_address=OIDC_ADDRESS,
    client_id=CLIENT_ID,
    token=getpass("Enter your access token: ")
)

Visit https://cloud-internal.h2o.ai/auth/get-platform-token to get your personal access token
Enter your access token: ········


In [21]:
app_service = appstore.AppServiceClient(ENDPOINT)
tag_service = appstore.TagServiceClient(ENDPOINT)


## Listing Apps

### All Apps

The clinent returnes a protobuf object, we then iterate through this to make it user-friendly.

In [31]:
apps_protobuf = app_service.ListApps(
    ctx = cp.ctx(),
    request = appstore.ListAppsRequest(
        offset = 0,
        limit = 1000,
        all_users = True
    ),
).apps

In [33]:
app_list = []

for i in apps_protobuf:
        
    app_list.append([i.title, i.version, i.id, i.owner])
    
apps = pd.DataFrame(
    app_list,
    columns = ['App Title', 'App Version', 'App ID',  'Owner']
)

apps = apps.sort_values(["App Title", "App Version"]).reset_index(drop=True)

apps

,App Title,App Version,App ID,Owner
0,AI Photo Restoration,1.0.0,edf72ad9-1976-4b7c-91f9-fd7a16408102,marcos.conde@h2o.ai
1,AI Unit Consumption,0.0.4,5017d664-43d4-4c3c-93b4-2de221db9115,tomasz.cichoszewski@h2o.ai
2,AI Unit Consumption,0.0.5,c93b1ff8-4a48-4850-a167-ac83b47cec02,tomasz.cichoszewski@h2o.ai
3,AI Unit Consumption,0.0.6,48d39a39-2b84-4477-b0d9-b6395a773b36,ashritha.eswaran@h2o.ai
4,Admin Analytics,1.0.0,2c13634a-4157-4097-a20d-36ac50ac9d79,michal.wysokinski@h2o.ai
...,...,...,...,...
218,User Usage Reporting,0.1.24,02efe015-ead3-44f0-89c9-bb1a39c67659,ashritha.eswaran@h2o.ai
219,Using H2O Drive,0.0.1-20220114135619,0c2a16e5-4976-4ce6-b8e5-252ad741aab8,michelle.tanco@h2o.ai
220,Vaccine NLP,0.1.6,86f78c48-dec9-42e2-b615-a9978ba62027,trushant.kalyanpur@h2o.ai
221,Wave App Tester,0.1.6,e3c97c97-5888-48d2-a3af-83f4784a8cd0,karthik.guruswamy@h2o.ai


### Find the app with the most versions

An app can have many versions. You can see how many versions an app has by running:

In [35]:
apps.groupby("App Title").size().reset_index(name="Count").sort_values("Count",ascending = False)

,App Title,Count
12,COVID-19 Hospital Occupancy Simulator,11
38,H2O AutoDoc,10
45,H2O Gene Mutation AI,8
53,H2O Olympics,8
39,H2O AutoInsights,7
...,...,...
65,"Hello, World!",1
19,Customer Churn Demo,1
18,Customer 360,1
17,Create AI Engine,1


### List all apps I own

To list all the apps that you own, the code is same as to how to list all the apps. The only change needed is to change all_users to False.

In [38]:
apps_protobuf = app_service.ListApps(
    ctx = cp.ctx(),
    request = appstore.ListAppsRequest(
        offset = 0,
        limit = 100,
        all_users = False 
    ),
).apps

In [40]:
my_apps = [f"{i.title} - {i.version} - {i.id}" for i in apps_protobuf]
my_apps

['Create AI Engine - 0.0.1-20220427112953 - cdfeb2f6-21fd-409c-b0ba-74b1b6c09b5e',
 'Customer Churn Demo - 1.2.0 - e5a4e715-3695-41e9-8c25-d3546f20eb87',
 'Theme Generator - 1.0.0 - 90cfefc2-a54b-47e8-9878-180a96535975',
 'Data Profiling - 1.0.1 - c090d7bd-22f1-45f2-9ebf-967c73849ab6',
 'Exploring Telco Churn - 1.0.0 - e5860b48-9ee5-4efc-ad26-7fb9f4df5929',
 'Telco Churn Predictions - 1.0.0 - dcb1abff-52b1-4829-a74e-c39ac5edff3f',
 'Hello, World! - 0.0.1 - c101c116-d050-455a-834b-eb4fc2ad67d7',
 'MLOps Prediction Demo - 0.0.1 - b7504ccf-aed5-466b-ba02-12bc922a7a5b',
 'Using H2O Drive - 0.0.1-20220114135619 - 0c2a16e5-4976-4ce6-b8e5-252ad741aab8',
 'App Store Management - 0.0.5 - b338ddf7-90ab-4d9f-96a6-2e2e79298a9a',
 'H2O AutoViz - 0.0.1 - 8d32c025-fdfe-4daa-8a40-5e69c17a75f3',
 'User Usage Reporting - 0.1.10 - a79653c9-a050-479e-8dcd-179c2e24f6a6',
 'H2O Feature Store - 1.0.0 - e450f814-6fab-40c7-b37a-b20a827c7e15']

## Edit an App
Make changes to an existing app version.

Set the variable `app_id` to an App ID found in the table above

In [41]:
app_id = 'c101c116-d050-455a-834b-eb4fc2ad67d7'

### Change the visibility of an app

Decide if the app should be visible in the App Store (2) or only in your My Apps page (1).

In [43]:
app_service.GetApp(
    ctx=cp.ctx(),
    request=appstore.GetAppRequest(
        id=app_id
    )
).app.visibility

2

In [44]:
app_service.UpdateApp(
    ctx = cp.ctx(),
    request = appstore.UpdateAppRequest(
        id = app_id,
        visibility = 1 
    ),
)

app {
  id: "c101c116-d050-455a-834b-eb4fc2ad67d7"
  create_time {
    seconds: 1647013089
    nanos: 353916000
  }
  update_time {
    seconds: 1647013089
    nanos: 353916000
  }
  owner: "michelle.tanco@h2o.ai"
  name: "ai.h2o.wave.my-first-app"
  title: "Hello, World!"
  description: "Show a card to the user explaining this is my first app."
  version: "0.0.1"
  bundle_location: "ai.h2o.wave.my-first-app.0.0.1.wave"
  visibility: PRIVATE
  tags {
    id: "5fa07cef-6ea0-4b1a-9cf2-ab0e2dc7e5ea"
    assign_time {
      seconds: 1649100347
      nanos: 98551000
    }
    name: "APP_DEV"
    title: "App Development"
    is_category: true
  }
  runtime_version: "deb10_py37_wlatest"
}

### View all tags assigned to an app

To view tags assigned to an app, pass in the ID of the app in the code below.

In [45]:
app = app_service.GetApp(
    ctx = cp.ctx(),
    request = appstore.GetAppRequest(
        id = app_id,
    ),
).app

app.tags

[id: "5fa07cef-6ea0-4b1a-9cf2-ab0e2dc7e5ea"
assign_time {
  seconds: 1649100347
  nanos: 98551000
}
name: "APP_DEV"
title: "App Development"
is_category: true
]

### List of tags

Here you can see the list of all the available tags and ID.

In [46]:
tag_list = tag_service.ListTags(
    ctx = cp.ctx(),
    request = appstore.ListTagsRequest(
    )
).tags

In [47]:
list_tags = []

for i in tag_list:       
    list_tags.append([i.title, i.name, i.id])
    
df_tags = pd.DataFrame(
   list_tags,
    columns = ['Tag Title', 'Tag Name', 'Tag ID']
)

df_tags

,Tag Title,Tag Name,Tag ID
0,Federal Government,FEDGOV,9948ad1d-2381-4c5d-84f5-636e11290fc7
1,H2O Tools,H2O_TOOLS,852c67fc-2aa6-4945-9f1e-8c2a1e1933bb
2,AI for Good,AI_FOR_GOOD,7d260cbb-8ef1-4df4-bcb0-ce1cfca54426
3,Exploratory Data Analysis,EDA,ae3afa33-de2e-428b-80f2-97d24bac9fb9
4,Machine Learning,MACHINE_LEARNING,a2bda56c-ae3b-4e41-9c6f-ee97d18ddeae
5,Explainability,EXPLAINABILITY,73bba919-c7a6-4ef0-a228-d1d3c78cbb86
6,Computer Vision,COMP_VISION,ad23aa48-58c3-41c5-85b2-734909ff27db
7,Forecasting,FORECASTING,4fc24fd4-c16f-4927-8cd2-10a1f24005df
8,Natural Language Processing,NLP,d66eb9e3-21e1-4d3c-b1c8-2014171e3a0b
9,Unsupervised Learning,UNSUPERVISED_LEARNING,76ed8390-cf38-486e-b3ca-4d1bb38130cc


### Add a new category tag to an app

To add a tag to an app, pass in the ID of the app and the ID of the tag you want to add! Here we are adding tag `Financial Services` to the app.

The ID of the tags can be found in the table where all the tags available are listed.

In [48]:
tag_service.AssignTag(
    ctx = cp.ctx(),
    request = appstore.AssignTagRequest(
        app_id = app_id,
        tag_id = '0385f27a-2155-11ec-9649-0a382865161b'),
)

### Remove a category tag from an app

To remove a tag from an app, again pass in the ID of the app and the ID of the tag you want to remove! Here we are removing tag `Financial Services` from the app.

In [49]:
tag_service.UnassignTag(
    ctx = cp.ctx(),
    request = appstore.UnassignTagRequest(
        app_id = app_id,
        tag_id ='0385f27a-2155-11ec-9649-0a382865161b'),
)

## App Instances

### Find the app with the most running instances
The following function lists the number the apps instances, running or suspended, of all the apps that are owned by all users.

In [54]:
all_instances = app_service.ListAppInstances(
    ctx=cp.ctx(),
    request=appstore.ListAppInstancesRequest(
        all_users=True,
        include_app_details=True
    ),
).instances

In [58]:
all_instance_list = [i.app_details.title for i in all_instances]

df_all_instances = pd.DataFrame(
    all_instance_list, 
    columns = ['App Name']
)

app_instance_counts = df_all_instances.groupby(['App Name']).size().reset_index(name='Count')
app_instance_counts = app_instance_counts.sort_values('Count', ascending=False).reset_index(drop = True)

app_instance_counts

,App Name,Count
0,H2O AutoDoc,41
1,H2O AI Cloud Notebook (alpha),33
2,H2O AutoInsights,32
3,H2O Hydrogen Torch,32
4,Model Validation,29
...,...,...
107,Medical Appointment No-Show,1
108,Medical Appointment no-show,1
109,Experiment Comparer,1
110,Notebooks to Wave Apps,1


### Start a new instance

Now lets start an instance of the app! Pass in the id of the app that you would like to start an instance of.

In [ ]:
app_id = '***'

In [ ]:
app_service.RunApp(
    ctx = cp.ctx(),
    request = appstore.RunAppRequest(
        id = app_id,
        visibility = 1
    ),
)

### List all instances I own
To list all the app instances that you own, the code is same as to how to list all the instances. The only change needed is to change all_users to False.

In [ ]:
instances_protobuf = app_service.ListAppInstances(
    ctx=cp.ctx(),
    request=appstore.ListAppInstancesRequest(
        all_users=False,
        include_app_details=True
    ),
).instances

In [ ]:
instance_list = []
for i in instances_protobuf:
    status = i.status
    if status == 5:
        status = 'Suspended'
    elif status == 3:
        status = 'Deployed'
    elif status == 2:
        status = 'Pending'
        
    instance_list.append(
        [i.owner, i.app_id, i.visibility, i.id, status]
    )
    
df_instances = pd.DataFrame(
    instance_list, 
    columns = ['App Name', 'App ID', 'App Visbility', 'Instance ID', 'Instance Status']
)

df_instances

### Pause and resume an instance

To pause an instance, pass in the ID of the instance and simply set suspend to True. To resume an instance, set suspend to False. 

In [ ]:
instance_id = '***'

In [ ]:
app_service.SetAppInstanceSuspension(
    ctx = cp.ctx(),
    request = appstore.SetAppInstanceSuspensionRequest(
        id = instance_id,
        suspend = True
    ),
)

### Delete an instance

When you no longer need an instance, you can terminate it. Remember to pass in the ID of the instance you want to delete!

In [ ]:
instance_id = '***'

In [ ]:
app_service.TerminateAppInstance(
    ctx = cp.ctx(),
    request = appstore.TerminateAppInstanceRequest(id = instance_id)
)